Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'E:/NLP/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

      # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

      # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

      # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.771812
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.193485
Minibatch accuracy: 18.8%
Validation accuracy: 25.8%
Minibatch loss at step 100: 1.394094
Minibatch accuracy: 43.8%
Validation accuracy: 61.3%
Minibatch loss at step 150: 0.634235
Minibatch accuracy: 81.2%
Validation accuracy: 74.4%
Minibatch loss at step 200: 0.907019
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 250: 1.348712
Minibatch accuracy: 62.5%
Validation accuracy: 78.1%
Minibatch loss at step 300: 0.354086
Minibatch accuracy: 87.5%
Validation accuracy: 78.5%
Minibatch loss at step 350: 0.468031
Minibatch accuracy: 93.8%
Validation accuracy: 77.9%
Minibatch loss at step 400: 0.263812
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 450: 0.808990
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 500: 0.691179
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

      # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(bias1,[1,2,2,1],[1,2,2,1],padding='SAME')
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        bias2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(bias2,[1,2,2,1],[1,2,2,1],padding='SAME')
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

      # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

      # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.731047
Minibatch accuracy: 12.5%
Validation accuracy: 12.2%
Minibatch loss at step 50: 1.815329
Minibatch accuracy: 12.5%
Validation accuracy: 42.0%
Minibatch loss at step 100: 1.377012
Minibatch accuracy: 37.5%
Validation accuracy: 59.8%
Minibatch loss at step 150: 0.403301
Minibatch accuracy: 87.5%
Validation accuracy: 73.0%
Minibatch loss at step 200: 0.920627
Minibatch accuracy: 75.0%
Validation accuracy: 77.7%
Minibatch loss at step 250: 1.179245
Minibatch accuracy: 62.5%
Validation accuracy: 78.4%
Minibatch loss at step 300: 0.360328
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 350: 0.476727
Minibatch accuracy: 93.8%
Validation accuracy: 79.2%
Minibatch loss at step 400: 0.280254
Minibatch accuracy: 93.8%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.878263
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 500: 0.544455
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

      # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
          [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        
        pool2 = tf.nn.avg_pool(bias1,[1,2,2,1],[1,2,2,1],padding='VALID')
        
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        
        pool4 = tf.nn.avg_pool(bias3,[1,2,2,1],[1,2,2,1],padding='VALID')
        
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

      # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

      # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.924498
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.669431
Minibatch accuracy: 43.8%
Validation accuracy: 53.0%
Minibatch loss at step 100: 1.088610
Minibatch accuracy: 62.5%
Validation accuracy: 65.5%
Minibatch loss at step 150: 0.884939
Minibatch accuracy: 68.8%
Validation accuracy: 69.9%
Minibatch loss at step 200: 1.267588
Minibatch accuracy: 56.2%
Validation accuracy: 70.6%
Minibatch loss at step 250: 1.158993
Minibatch accuracy: 75.0%
Validation accuracy: 74.7%
Minibatch loss at step 300: 0.617617
Minibatch accuracy: 87.5%
Validation accuracy: 76.9%
Minibatch loss at step 350: 0.867603
Minibatch accuracy: 87.5%
Validation accuracy: 75.0%
Minibatch loss at step 400: 0.356127
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 450: 0.780512
Minibatch accuracy: 81.2%
Validation accuracy: 76.8%
Minibatch loss at step 500: 0.896916
Minibatch accuracy: 87.5%
Validation accuracy: 78.2%
M

Validation accuracy: 86.3%
Minibatch loss at step 4550: 0.368674
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 4600: 0.401606
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 4650: 0.929840
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 4700: 0.506212
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 4750: 0.974712
Minibatch accuracy: 68.8%
Validation accuracy: 86.1%
Minibatch loss at step 4800: 0.520008
Minibatch accuracy: 81.2%
Validation accuracy: 86.3%
Minibatch loss at step 4850: 0.332478
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 4900: 0.190567
Minibatch accuracy: 93.8%
Validation accuracy: 86.4%
Minibatch loss at step 4950: 0.143275
Minibatch accuracy: 100.0%
Validation accuracy: 86.6%
Minibatch loss at step 5000: 1.055682
Minibatch accuracy: 75.0%
Validation accuracy: 85.7%
Minibatch loss at step 5050: 0.329705
Minibatch accuracy: 93.8

Validation accuracy: 87.8%
Minibatch loss at step 9050: 0.360630
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 9100: 0.346226
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 9150: 0.810940
Minibatch accuracy: 68.8%
Validation accuracy: 87.4%
Minibatch loss at step 9200: 0.186155
Minibatch accuracy: 93.8%
Validation accuracy: 87.4%
Minibatch loss at step 9250: 0.644595
Minibatch accuracy: 75.0%
Validation accuracy: 87.7%
Minibatch loss at step 9300: 1.032110
Minibatch accuracy: 81.2%
Validation accuracy: 87.8%
Minibatch loss at step 9350: 0.332949
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 9400: 0.352087
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 9450: 0.351623
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 9500: 0.209071
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 9550: 0.252361
Minibatch accuracy: 87.5%

Minibatch loss at step 13500: 0.394998
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 13550: 0.593683
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 13600: 0.471683
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 13650: 0.480290
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 13700: 0.228940
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 13750: 0.772769
Minibatch accuracy: 75.0%
Validation accuracy: 88.6%
Minibatch loss at step 13800: 0.070751
Minibatch accuracy: 100.0%
Validation accuracy: 88.4%
Minibatch loss at step 13850: 0.223331
Minibatch accuracy: 93.8%
Validation accuracy: 88.3%
Minibatch loss at step 13900: 0.207688
Minibatch accuracy: 93.8%
Validation accuracy: 88.6%
Minibatch loss at step 13950: 0.481331
Minibatch accuracy: 81.2%
Validation accuracy: 88.4%
Minibatch loss at step 14000: 0.033714
Minibatch accuracy: 100.0%
Validation ac

Minibatch loss at step 17950: 0.561763
Minibatch accuracy: 81.2%
Validation accuracy: 88.8%
Minibatch loss at step 18000: 0.413315
Minibatch accuracy: 81.2%
Validation accuracy: 88.3%
Minibatch loss at step 18050: 0.373883
Minibatch accuracy: 87.5%
Validation accuracy: 88.9%
Minibatch loss at step 18100: 0.696660
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 18150: 0.216314
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 18200: 0.447413
Minibatch accuracy: 93.8%
Validation accuracy: 89.1%
Minibatch loss at step 18250: 0.598159
Minibatch accuracy: 75.0%
Validation accuracy: 88.9%
Minibatch loss at step 18300: 0.134365
Minibatch accuracy: 100.0%
Validation accuracy: 88.6%
Minibatch loss at step 18350: 0.287170
Minibatch accuracy: 93.8%
Validation accuracy: 89.2%
Minibatch loss at step 18400: 0.757796
Minibatch accuracy: 75.0%
Validation accuracy: 88.7%
Minibatch loss at step 18450: 0.407793
Minibatch accuracy: 87.5%
Validation acc

+ add Dropout,learning rate

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta_regul = 1e-3
drop_out = 0.5

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)

      # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
          [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer5_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
  # Model.
    def model(data,keep_prob):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        
        pool2 = tf.nn.avg_pool(bias1,[1,2,2,1],[1,2,2,1],padding='VALID')
        
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        
        pool4 = tf.nn.avg_pool(bias3,[1,2,2,1],[1,2,2,1],padding='VALID')
        
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        drop5 = tf.nn.dropout(hidden5,keep_prob)
        hidden6 = tf.nn.relu(tf.matmul(hidden5, layer4_weights) + layer4_biases)
        drop6 = tf.nn.dropout(hidden6,keep_prob)
        return tf.matmul(drop6, layer5_weights) + layer5_biases
  
  # Training computation.
    logits = model(tf_train_dataset,drop_out)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

      # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05,global_step,1000,0.85,staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

      # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset,1.0))
    test_prediction = tf.nn.softmax(model(tf_test_dataset,1.0))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.500009
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.201375
Minibatch accuracy: 18.8%
Validation accuracy: 38.0%
Minibatch loss at step 100: 1.805668
Minibatch accuracy: 37.5%
Validation accuracy: 58.7%
Minibatch loss at step 150: 1.127190
Minibatch accuracy: 62.5%
Validation accuracy: 62.3%
Minibatch loss at step 200: 1.367682
Minibatch accuracy: 56.2%
Validation accuracy: 63.3%
Minibatch loss at step 250: 1.373265
Minibatch accuracy: 56.2%
Validation accuracy: 68.8%
Minibatch loss at step 300: 0.727963
Minibatch accuracy: 87.5%
Validation accuracy: 73.9%
Minibatch loss at step 350: 0.857724
Minibatch accuracy: 81.2%
Validation accuracy: 74.2%
Minibatch loss at step 400: 0.598572
Minibatch accuracy: 75.0%
Validation accuracy: 76.0%
Minibatch loss at step 450: 1.183064
Minibatch accuracy: 68.8%
Validation accuracy: 74.4%
Minibatch loss at step 500: 1.221983
Minibatch accuracy: 87.5%
Validation accuracy: 77.2%
Mi

Validation accuracy: 85.9%
Minibatch loss at step 4550: 0.435453
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 4600: 0.386717
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 4650: 1.013988
Minibatch accuracy: 87.5%
Validation accuracy: 84.8%
Minibatch loss at step 4700: 0.512556
Minibatch accuracy: 81.2%
Validation accuracy: 85.9%
Minibatch loss at step 4750: 1.204422
Minibatch accuracy: 68.8%
Validation accuracy: 85.9%
Minibatch loss at step 4800: 0.555512
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 4850: 0.436879
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 4900: 0.351243
Minibatch accuracy: 93.8%
Validation accuracy: 86.0%
Minibatch loss at step 4950: 0.258668
Minibatch accuracy: 93.8%
Validation accuracy: 85.8%
Minibatch loss at step 5000: 1.197379
Minibatch accuracy: 68.8%
Validation accuracy: 85.6%
Minibatch loss at step 5050: 0.279131
Minibatch accuracy: 93.8%

Validation accuracy: 86.9%
Minibatch loss at step 9050: 0.486596
Minibatch accuracy: 81.2%
Validation accuracy: 87.0%
Minibatch loss at step 9100: 0.355452
Minibatch accuracy: 93.8%
Validation accuracy: 87.0%
Minibatch loss at step 9150: 0.705886
Minibatch accuracy: 68.8%
Validation accuracy: 87.0%
Minibatch loss at step 9200: 0.170454
Minibatch accuracy: 93.8%
Validation accuracy: 87.2%
Minibatch loss at step 9250: 1.277819
Minibatch accuracy: 68.8%
Validation accuracy: 86.9%
Minibatch loss at step 9300: 1.063855
Minibatch accuracy: 81.2%
Validation accuracy: 86.9%
Minibatch loss at step 9350: 0.328239
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 9400: 0.312563
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 9450: 0.370232
Minibatch accuracy: 93.8%
Validation accuracy: 86.9%
Minibatch loss at step 9500: 0.190596
Minibatch accuracy: 100.0%
Validation accuracy: 87.0%
Minibatch loss at step 9550: 0.268773
Minibatch accuracy: 87.5

Minibatch loss at step 13500: 0.385611
Minibatch accuracy: 81.2%
Validation accuracy: 87.3%
Minibatch loss at step 13550: 0.661020
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 13600: 0.339552
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 13650: 0.780575
Minibatch accuracy: 75.0%
Validation accuracy: 87.6%
Minibatch loss at step 13700: 0.406991
Minibatch accuracy: 93.8%
Validation accuracy: 87.4%
Minibatch loss at step 13750: 0.958009
Minibatch accuracy: 75.0%
Validation accuracy: 87.2%
Minibatch loss at step 13800: 0.019462
Minibatch accuracy: 100.0%
Validation accuracy: 87.2%
Minibatch loss at step 13850: 0.333406
Minibatch accuracy: 93.8%
Validation accuracy: 87.4%
Minibatch loss at step 13900: 0.188585
Minibatch accuracy: 93.8%
Validation accuracy: 87.4%
Minibatch loss at step 13950: 0.456456
Minibatch accuracy: 81.2%
Validation accuracy: 87.6%
Minibatch loss at step 14000: 0.064533
Minibatch accuracy: 100.0%
Validation ac

Minibatch loss at step 17950: 0.588948
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 18000: 0.557760
Minibatch accuracy: 81.2%
Validation accuracy: 87.6%
Minibatch loss at step 18050: 0.532236
Minibatch accuracy: 81.2%
Validation accuracy: 87.8%
Minibatch loss at step 18100: 0.982737
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 18150: 0.441763
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 18200: 0.665092
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 18250: 0.809627
Minibatch accuracy: 75.0%
Validation accuracy: 87.8%
Minibatch loss at step 18300: 0.277716
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 18350: 0.369453
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 18400: 0.803077
Minibatch accuracy: 68.8%
Validation accuracy: 87.8%
Minibatch loss at step 18450: 0.275614
Minibatch accuracy: 87.5%
Validation accu